In [1]:
%runfile ../General-Tools/LazyList.sage
%runfile ../TS-Methods/MonoidTS.sage
%runfile ../TS-Methods/RingTS.sage
%runfile ../Sequence-Methods/Berlekamp.sage
%runfile ../TS-Methods/ComplexTS.sage

In [2]:
TS = ComplexTannakianSymbols()
ts = TS.parseSymbol

In [3]:
import pickle
import json

In [4]:
import random

maxEvenDim = 10
maxOddDim = 10
elementRange = (1, 5)

precision = 20

length = 50000

data = [None] * length
symbols = [None] * length

def printProgress(i, max):
    sys.stdout.write("\r%d (%d%%)" % (i, 100 * i / max))
    sys.stdout.flush()

In [6]:

random.seed(42)
for i in range(length):
    evendim = random.randint(0, maxEvenDim)
    odddim  = random.randint(0, maxOddDim)
    sym = []
    for j in range(evendim):
        sym.append([random.randint(*elementRange), 1])
    for j in range(odddim):
        sym.append([random.randint(*elementRange), -1])
    symbol = TS.createElement(sym)
    data[i] = (symbol.getPointCounts().toList(precision), symbol.augmentation())
    symbols[i] = symbol
    
    if i != 0 and (i % 100) == 0:
        printProgress(i, length)

printProgress(length, length)

50000 (100%)

In [7]:
with open('recursionData2.json', 'w') as f:
    json.dump(map(lambda (l, r): (map(float, l), int(r)), data), f)

In [8]:
with open('recursionData2.json') as f:
    data2 = map(lambda (l, r): (map(float, l), int(r)), json.load(f))

In [22]:
augmentation_range = maxOddDim + maxEvenDim
toIndex = lambda aug: maxOddDim + aug
toAugmentation = lambda ind: aug - maxOddDim

In [20]:
training_size = 45000

In [21]:
testing_size = 5000

In [52]:
properData = map(lambda (coeffs, aug): (coeffs, [0]*(toIndex(aug)) + [1] + [0]*(augmentation_range - toIndex(aug))), data2)
properData = data2

In [53]:
training_data = properData[0:training_size]

In [54]:
testing_data = properData[training_size:training_size + testing_size]

In [55]:
import tensorflow as tf
import numpy as np

In [128]:
x = tf.placeholder(tf.int32, [None, precision], name="x")

In [129]:
W = tf.Variable(tf.zeros([precision, 1], tf.float32), name="W")

In [130]:
y = tf.matmul(tf.cast(x, tf.float32), W, name="y")

In [131]:
y_ = tf.placeholder(tf.int32, [None, 1], name="y_")

In [132]:
# cross entropy might not be ideal
cost = tf.square(tf.cast(tf.round(y), tf.int32) - y_)

In [134]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

ValueError: Invalid type tf.int32 for Square_3:0, expected: [tf.float32, tf.float64, tf.float16].

In [109]:
init = tf.global_variables_initializer()

In [110]:
sess = tf.Session()

In [111]:
sess.run(init)

In [112]:
printProgress(0, training_size)
for i, (bellCoeffs, aug) in enumerate(training_data):
    sess.run(train_step, feed_dict={x: [bellCoeffs], y_: [[aug]]})
    
    if sess.run(tf.reduce_sum(tf.cast(tf.is_nan(W), tf.float32))) > 0:
        print 
        print "became nan at data[", i, "]"
        break
        
    if (i + 1) % 50 == 0:
        printProgress(i + 1, training_size)

0 (0%)
became nan at data[ 3 ]


In [94]:
testing_data_input  = map(lambda (bellCoeffs, aug): bellCoeffs, testing_data)
testing_data_output = map(lambda (bellCoeffs, aug): [aug], testing_data)

correct_prediction = tf.equal(tf.round(y), tf.round(y_))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print sess.run(accuracy, feed_dict={x: testing_data_input, y_: testing_data_output})

0.0


In [78]:
sess.close()

In [86]:
sess.run(cost, feed_dict={x: [training_data[0][0]], y_: [[training_data[0][1]]]})

nan

In [72]:
training_data[16][0]

[1.0,
 30.0,
 152.0,
 762.0,
 3800.0,
 18930.0,
 94352.0,
 470682.0,
 2349800.0,
 11737410.0,
 58650752.0,
 293141802.0,
 1465367000.0,
 7325796690.0,
 36625843952.0,
 183119752122.0,
 915570259400.0,
 4577765596770.0,
 22888570489952.0,
 114442079181642.0]

In [73]:
sess.run(W)

array([[  4.50204102e+03],
       [  3.08601288e+02],
       [  1.04920605e+03],
       [  4.10238672e+03],
       [  1.71231055e+04],
       [  7.41537969e+04],
       [  3.29424656e+05],
       [  1.49285875e+06],
       [  6.87696350e+06],
       [  3.21189820e+07],
       [  1.51769168e+08],
       [  7.24232448e+08],
       [  3.48474957e+09],
       [  1.68841974e+10],
       [  8.22801531e+10],
       [  4.02893570e+11],
       [  1.98059767e+12],
       [  9.76798771e+12],
       [  4.83012819e+13],
       [  2.39357185e+14]], dtype=float32)